In [83]:
import os
import sys

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from sklearnex import patch_sklearn
patch_sklearn()

import heteroscedastic_uncertainty_regressor as hr


import sklearn
from sklearn.preprocessing import KBinsDiscretizer
import datetime

import plot_tools
import dataset_stats as ds

import scipy.special
import scipy.stats

import statsmodels.api as sm

import importlib

importlib.reload(hr)
importlib.reload(plot_tools)
importlib.reload(ds)

%matplotlib qt

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [48]:
VERSION = "v1"
MODEL_TYPE = "LOWER_BAND"

pdata_folder = os.path.abspath("./../processed_data/chorus_neural_network/")
rbsp_chorus_folder = os.path.join(pdata_folder, "observed_chorus")
output_folder = os.path.join(pdata_folder, "models", VERSION)


dataset = np.load(
    file=os.path.join(output_folder, rf"total_dataset_{VERSION}_{MODEL_TYPE}.npz")
)

X_conditional = dataset["X_conditional"]
X_convolutional = dataset["X_convolutional"]
y = dataset["y"]

print(y.shape)
print(X_conditional.shape)
print(X_convolutional.shape)

greater_than_zero = y > 0
X_conditional = X_conditional[greater_than_zero, :]
X_convolutional = X_convolutional[greater_than_zero, :]
y = y[greater_than_zero]

dataset.close()

t_data = X_conditional[:, -1]
X_conditional = X_conditional[:, :-1]

(3393325,)
(3393325, 7)
(3393325, 512)


In [54]:
print(np.min(y / X_conditional[:, -2]))
print(np.max(y / X_conditional[:, -2]))

0.0035322735863470237
19908816.024684824


In [81]:
P = y**(1/3)


In [ ]:
plot_tools.plot_2d_heatmap(X_conditional[:, 0] * X_conditional[:, 1],
                           X_conditional[:, 0] * X_conditional[:, 2],
                           y,
                           bins=20,
                           xlim=(-10, 10),
                           ylim=(-10, 10),
                           xtitle="X",
                           ytitle="Y",
                           ztitle="Average Chorus Amplitude (pT)",
                           title="Average Chorus Amplitude vs X vs Y",
                           norm="symlog",
                           density_norm="log",
                           plot_density = True)

In [82]:
ax0 = sns.displot(P, log_scale=True)
ax0.set(ylabel='N', xlabel='Chorus (pT)', title='Training Set')
plt.tight_layout()

In [7]:
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile', subsample=None)

discretizer.fit(y[:, np.newaxis])

y_discretized = discretizer.transform(y[:, np.newaxis])
bin_edges = discretizer.bin_edges_
print(bin_edges)
print(y_discretized)

[array([1.00000178e+00, 1.95501273e+00, 3.87673562e+00, 8.45249222e+00,
        2.52017115e+01, 4.30599589e+05])                               ]
[[4.]
 [3.]
 [3.]
 ...
 [2.]
 [1.]
 [3.]]


In [4]:
ax0 = sns.displot(y)
ax0.set(ylabel='N', xlabel='Chorus (pT)', title='Training Set')
plt.tight_layout()

KeyboardInterrupt: 

In [45]:
ax0 = sns.displot(transformed_y)
ax0.set(ylabel='N', xlabel='After box-cox', title='Box Cox Transformed')
plt.tight_layout()

In [46]:
ax0 = sns.displot(np.log(y + 1))
ax0.set(ylabel='N', xlabel='After box-cox', title='Log (y + 1)')
plt.tight_layout()

In [3]:
# Figure out and apply L filter to increase percentage of non-zero elements
print("\nTotal dataset\n")

ds.print_dataset_stats(y, {"count", "% 0", "% != 0"})
ds.print_dataset_stats(y, {"count"}, greater_than=10)


Total dataset


Dataset Statistics:
--------------------------------------------------
Percentage of Non-Zeros  : 100.00%
Percentage of Zeros      : 0.00%
Count (non-NaN)          : 411594.0000
--------------------------------------------------

Dataset Statistics:
(Values greater than 10)
--------------------------------------------------
Count (non-NaN)          : 149244.0000
--------------------------------------------------


In [ ]:
ax0 = sns.displot(y)
ax0.set(ylabel='N', xlabel='Chorus (pT)', title='Training Set')
plt.tight_layout()

In [ ]:
plot_tools.plot_2d_heatmap(X_conditional[:, 0] * X_conditional[:, 1],
                           X_conditional[:, 0] * X_conditional[:, 2],
                           y,
                           bins=20,
                           xlim=(-10, 10),
                           ylim=(-10, 10),
                           xtitle="X",
                           ytitle="Y",
                           ztitle="Average Chorus Amplitude (pT)",
                           title="Average Chorus Amplitude vs X vs Y",
                           norm="symlog",
                           density_norm="log",
                           plot_density = True)

In [4]:
# Preprocess dataset
RS_conditional = sklearn.preprocessing.RobustScaler(copy=False)
RS_conditional.fit(X_conditional)
transformed_X_conditional = RS_conditional.transform(X_conditional)

RS_convolutional = sklearn.preprocessing.RobustScaler(copy=False)
RS_convolutional.fit(X_convolutional)
transformed_X_convolutional = RS_convolutional.transform(X_convolutional)

In [5]:
ref_date = datetime.datetime(year=2012, month=1, day=1)
g_data = np.zeros_like(y)
for i, t in enumerate(t_data):

    dt = datetime.datetime.fromtimestamp(t)
    del_dt = dt - ref_date
    g_data[i] = del_dt.days

In [ ]:
# Initialize model and train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models, transformers = hr.cross_validate_model(
    X_conditional=transformed_X_conditional,
    X_convolutional=transformed_X_convolutional,
    y=np.log(y),
    g_data=g_data,
    bias_model_class=hr.BiasGenerator,
    variance_model_class=hr.VarianceGenerator,
    bias_learning_rate=2e-5,
    variance_learning_rate=1e-4,
    cond_input_dim=6,
    hidden_dim=256,
    noise_dim=2,
    n_splits=5,
    n_epochs=2,
    batch_size=128,
    weight_decay=1e-6,
    optim_betas=(0.9, 0.99),
    device="cuda",
    output_folder="./../processed_data/chorus_neural_network/models/v1/trained_models/"
)



In [48]:
trained_model_folder = os.path.join(output_folder, "trained_models")

saved_folder = os.path.abspath(trained_model_folder)

for m in range(1):
    bias_model=hr.BiasGenerator(input_dim=6, hidden_dim=256)
    noise=torch.randn((10000, 2)).to(device)
    bias_model.load_state_dict(torch.load(os.path.join(saved_folder, f"bias_model_{m}.pth")), strict=False)
    bias_model.to("cuda")
    bias_model.eval()

    var_model=hr.VarianceGenerator(input_dim=6, hidden_dim=256)
    var_model.load_state_dict(torch.load(os.path.join(saved_folder, f"var_model_{m}.pth")), strict=False)
    var_model.to("cuda")
    var_model.eval()
    # Evaluate and plot
    with torch.no_grad():

        t_X_cond_np = torch.from_numpy(transformed_X_conditional[0:10000, :]).to("cuda").type(torch.float32)
        t_X_conv_np = torch.from_numpy(transformed_X_convolutional[0:10000, np.newaxis, :]).to("cuda").type(torch.float32)
        pred_mean = bias_model(t_X_cond_np, t_X_conv_np, noise)
        pred_var = var_model(t_X_cond_np, pred_mean)

    break

print(pred_mean, pred_var)

tensor([[117.3898],
        [126.7808],
        [107.5210],
        ...,
        [ 53.1432],
        [171.9191],
        [133.6663]], device='cuda:0') tensor([[3.7937],
        [4.0902],
        [3.4280],
        ...,
        [2.1027],
        [5.4533],
        [4.1535]], device='cuda:0')


In [49]:
plt.errorbar(x=np.sqrt(y[:10000].flatten()),
             y=np.sqrt(pred_mean[:10000].cpu().numpy().flatten()),
             fmt=".")
plt.xscale("symlog")
plt.yscale("symlog")
plt.xlim(0, 1000)
plt.ylim(0, 1000)

(0.0, 1000.0)